In [3]:
import mysql
import mysql.connector
import pandas as pd
import joblib
from prophet import Prophet

# Load the sales dataset
sales_df = pd.read_csv("SALES.csv", parse_dates=["Order_Date"])
sales_agg = sales_df.groupby("Order_Date")["Quantity"].sum().reset_index()
sales_agg.columns = ["ds", "y"]

# Train Prophet model
model = Prophet()
model.fit(sales_agg)

# Save the model
joblib.dump(model, "prophet_model.pkl")

# Load the saved model
loaded_model = joblib.load("prophet_model.pkl")

# Create future dataframe for forecasting
future = loaded_model.make_future_dataframe(periods=30)
forecast = loaded_model.predict(future)

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="VIJAY@25",
    database="SmartInventory"
)
cursor = conn.cursor()

# Create table if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS sales_forecast (
    id INT AUTO_INCREMENT PRIMARY KEY,
    date DATE,
    forecast_value FLOAT
);
""")

# Insert forecasted data into MySQL
for index, row in forecast.iterrows():
    cursor.execute("INSERT INTO sales_forecast (date, forecast_value) VALUES (%s, %s)", (row["ds"], row["yhat"]))

# Commit and close connection
conn.commit()
cursor.close()
conn.close()
print("Forecast results stored in MySQL successfully!")


08:48:51 - cmdstanpy - INFO - Chain [1] start processing
08:48:51 - cmdstanpy - INFO - Chain [1] done processing


Forecast results stored in MySQL successfully!


In [ ]:
import numpy as np
import pandas as pd
import joblib

# Load trained Q-table
q_table = joblib.load("optimized_q_table.pkl")

# Load inventory data
inventory_df = pd.read_csv("updated_inventory_data.csv")

# Ensure State_Num is correctly assigned
assert inventory_df["State_Num"].isin([0, 1, 2]).all(), "Error: State_Num contains unexpected values!"

# Apply Q-table to predict Optimal_Restocking_Time
inventory_df["Optimal_Restocking_Time"] = inventory_df["State_Num"].apply(lambda state: np.argmax(q_table[state]))

# Save updated dataset
inventory_df.to_csv("updated_inventory_with_rl.csv", index=False)

print("✔️ Q-learning model applied successfully!")
print(inventory_df.head())


In [5]:
def classify_state(stock, reorder_level):
    if stock <= reorder_level:
        return 0  # Low Stock
    elif stock <= reorder_level * 2:
        return 1  # Medium Stock
    else:
        return 2  # High Stock

inventory_data_rl["State_Num"] = inventory_data_rl.apply(lambda row: classify_state(row["Current_Stock"], row["Reorder_Level"]), axis=1)

# Now, apply RL model
inventory_data_rl["Optimal_Restocking_Time"] = inventory_data_rl["State_Num"].apply(get_optimal_action)


In [7]:
inventory_data_rl.head()

,Store_Key,Product_Key,Initial_Stock,Current_Stock,Reorder_Level,Last_Restock_Date,Supplier,State_Num,Optimal_Restocking_Time
0,1,1,72,72,14,2024-11-05,Supplier B,2,1
1,1,2,170,170,34,2024-12-09,Supplier B,2,1
2,1,3,383,383,76,2024-10-18,Supplier B,2,1
3,1,4,284,284,56,2024-10-22,Supplier B,2,1
4,1,5,135,135,27,2024-11-26,Supplier D,2,1


In [8]:
print("Unique State_Num values in dataset:", inventory_data_rl["State_Num"].unique())

Unique State_Num values in dataset: [2 0 1]


In [7]:
# Create 'State' column (example based on inventory levels)
inventory_data_rl['State'] = (
    inventory_data_rl['Current_Stock'] - inventory_data_rl['Reorder_Level']
).apply(lambda x: "Low" if x <= 0 else "High")


In [8]:
# Map states to numerical values (adjust as needed)
state_mapping = {"Low": 0, "High": 1}
inventory_data_rl['State_Num'] = inventory_data_rl['State'].map(state_mapping)


In [9]:
import numpy as np

# Function to find the optimal action
def get_optimal_action(state_num):
    if state_num < len(rl_model):  # Ensure state_num is within range
        return np.argmax(rl_model[state_num])  # Action with max Q-value
    else:
        return "Invalid State"

# Apply function to determine the optimal restocking action
inventory_data_rl["Optimal_Restocking_Time"] = inventory_data_rl["State_Num"].apply(get_optimal_action)


In [10]:
print(inventory_data_rl.head())


   Store_Key  Product_Key  Initial_Stock  Current_Stock  Reorder_Level  \
0          1            1             72             72             14   
1          1            2            170            170             34   
2          1            3            383            383             76   
3          1            4            284            284             56   
4          1            5            135            135             27   

  Last_Restock_Date    Supplier State  State_Num  Optimal_Restocking_Time  
0        2024-11-05  Supplier B  High          1                        1  
1        2024-12-09  Supplier B  High          1                        1  
2        2024-10-18  Supplier B  High          1                        1  
3        2024-10-22  Supplier B  High          1                        1  
4        2024-11-26  Supplier D  High          1                        1  


In [2]:
import pandas as pd
inventry_df = pd.read_csv("updated_inventory_data.csv")
inventry_df.head()

,Store_Key,Product_Key,Initial_Stock,Current_Stock,Reorder_Level,Last_Restock_Date,Supplier,State,State_Num,Optimal_Restocking_Time,Action_Description
0,1,1,72,72,14,2024-11-05,Supplier B,High,1,1,Restock Immediately
1,1,2,170,170,34,2024-12-09,Supplier B,High,1,1,Restock Immediately
2,1,3,383,383,76,2024-10-18,Supplier B,High,1,1,Restock Immediately
3,1,4,284,284,56,2024-10-22,Supplier B,High,1,1,Restock Immediately
4,1,5,135,135,27,2024-11-26,Supplier D,High,1,1,Restock Immediately


In [11]:
print(f"Q-Values for State 1: {rl_model[1]}")


Q-Values for State 1: [-3.  0.  0.]


In [12]:
action_mapping = {0: "Do Nothing", 1: "Restock Immediately", 2: "Monitor"}
inventory_data_rl["Action_Description"] = inventory_data_rl["Optimal_Restocking_Time"].map(action_mapping)
print(inventory_data_rl.head())


   Store_Key  Product_Key  Initial_Stock  Current_Stock  Reorder_Level  \
0          1            1             72             72             14   
1          1            2            170            170             34   
2          1            3            383            383             76   
3          1            4            284            284             56   
4          1            5            135            135             27   

  Last_Restock_Date    Supplier State  State_Num  Optimal_Restocking_Time  \
0        2024-11-05  Supplier B  High          1                        1   
1        2024-12-09  Supplier B  High          1                        1   
2        2024-10-18  Supplier B  High          1                        1   
3        2024-10-22  Supplier B  High          1                        1   
4        2024-11-26  Supplier D  High          1                        1   

    Action_Description  
0  Restock Immediately  
1  Restock Immediately  
2  Restock Immedi

In [14]:
def get_optimal_action(state_num):
    # Check if state_num is within the bounds of the Q-table
    if state_num >= len(rl_model):
        return "Unknown State"
    
    # Retrieve the Q-values for the given state_num
    q_values = rl_model[state_num]
    optimal_action = q_values.argmax()
    return optimal_action


In [15]:
# Apply the function to determine the optimal action for each row
inventory_data_rl["Optimal_Restocking_Time"] = inventory_data_rl["State_Num"].apply(get_optimal_action)

# Map the action to human-readable descriptions
action_mapping = {0: "Do Nothing", 1: "Restock Immediately", 2: "Monitor"}
inventory_data_rl["Action_Description"] = inventory_data_rl["Optimal_Restocking_Time"].map(action_mapping)

# Display the updated DataFrame
print(inventory_data_rl.head())


   Store_Key  Product_Key  Initial_Stock  Current_Stock  Reorder_Level  \
0          1            1             72             72             14   
1          1            2            170            170             34   
2          1            3            383            383             76   
3          1            4            284            284             56   
4          1            5            135            135             27   

  Last_Restock_Date    Supplier State  State_Num  Optimal_Restocking_Time  \
0        2024-11-05  Supplier B  High          1                        1   
1        2024-12-09  Supplier B  High          1                        1   
2        2024-10-18  Supplier B  High          1                        1   
3        2024-10-22  Supplier B  High          1                        1   
4        2024-11-26  Supplier D  High          1                        1   

    Action_Description  
0  Restock Immediately  
1  Restock Immediately  
2  Restock Immedi

In [1]:
import pandas as pd

# Example: Load your inventory dataset
inventory_data_rl = pd.read_csv("Inventory1.csv") # Adjust the path





In [2]:
from IPython.display import FileLink

# Generate a download link for the file
FileLink("updated_inventory_data.csv")


C:\Users\vijayabalan\updated_inventory_data.csv

In [6]:
import joblib

q_table = joblib.load("C:/Users/vijayabalan/Downloads/optimized_q_table.pkl")
print(type(q_table))
print(q_table)


<class 'numpy.ndarray'>
[[ 0.          0.          0.        ]
 [-3.          0.          0.        ]
 [-3.          0.          0.        ]
 [-3.         29.0090933   0.        ]
 [-3.          0.          0.        ]
 [-3.         94.87719409  0.        ]
 [-3.         98.6554149   0.        ]
 [-5.1        95.49852911  0.        ]
 [-5.1        98.12391594 29.09693504]
 [-3.         99.59953625  0.        ]
 [-1.73205751 99.88004492 29.1677346 ]
 [-3.         98.95526201  0.        ]
 [-3.         99.65709863  0.        ]
 [12.00105864 99.97481638 49.11015208]
 [-6.57       98.68628382 28.9549638 ]
 [44.92137158 28.77917593 99.99999883]
 [65.40862015 99.99999895 87.20250265]
 [57.12715503 99.99999931 89.10689745]
 [67.63114163 99.99999925 89.41739168]
 [55.9029248  99.99999933 76.63431293]
 [78.45922221 99.99999927 70.47478127]
 [80.10898961 99.99999933 83.68604572]
 [86.58206966 99.99999934 81.43156704]
 [88.68356068 99.99999934 79.75162659]
 [86.83286803 99.99999924 77.70786272]
 

In [ ]:
import numpy as np

# Define stock level states
MAX_STOCK = 100  
REORDER_LEVEL = 30  
OVERSTOCK_LEVEL = 80  

# Define actions: 0 = No restock, 1 = Medium restock, 2 = Large restock
ACTIONS = [0, 1, 2]

# Q-learning parameters
ALPHA = 0.1  # Learning rate
GAMMA = 0.9  # Discount factor
EPSILON = 0.1  # Exploration rate
EPISODES = 1000  

# Initialize Q-table
q_table = np.zeros((MAX_STOCK + 1, len(ACTIONS)))  # (Stock levels, Actions)

# Function to get reward based on stock level
def get_reward(stock, action):
    if action == 0:  # No restock
        new_stock = max(stock - np.random.randint(5, 15), 0)  # Simulate demand reduction
    elif action == 1:  # Medium restock
        new_stock = stock + np.random.randint(10, 20)
    else:  # Large restock
        new_stock = stock + np.random.randint(20, 30)

    new_stock = min(new_stock, MAX_STOCK)  # ✅ Ensure stock does not exceed max limit

    # Reward system
    if new_stock < REORDER_LEVEL:
        reward = -10  # Penalty for low stock
    elif new_stock > OVERSTOCK_LEVEL:
        reward = -5   # Penalty for overstocking
    else:
        reward = 10   # Reward for optimal stock level

    return reward, new_stock

# Train Q-learning model
for episode in range(EPISODES):
    stock = np.random.randint(0, MAX_STOCK)  # Start with random stock level
    
    while stock > 0:  # Run until stock is depleted
        # Choose action (ε-greedy strategy)
        if np.random.rand() < EPSILON:
            action = np.random.choice(ACTIONS)  # Explore
        else:
            action = np.argmax(q_table[stock])  # Exploit best action

        # Get reward and new state
        reward, new_stock = get_reward(stock, action)

        # Update Q-table using Bellman equation
        q_table[stock, action] = (1 - ALPHA) * q_table[stock, action] + \
                                 ALPHA * (reward + GAMMA * np.max(q_table[new_stock]))

        stock = new_stock  # Move to new state
        
        if stock == 0:  # Stop episode if stock is completely depleted
            break

# ✅ Save trained model
import joblib
joblib.dump(q_table, "reinforcement_learning_q_table.pkl")
print("✔️ Q-learning model saved successfully!")

# ✅ Load the model (for verification)
q_table_loaded = joblib.load("reinforcement_learning_q_table.pkl")
print("✔️ Q-table loaded successfully! Shape:", q_table_loaded.shape)


In [ ]:
import mysql.connector
import pandas as pd
import joblib
from prophet import Prophet

# Load the sales dataset
sales_df = pd.read_csv("/mnt/data/Sales.csv", parse_dates=["Order_Date"])
sales_agg = sales_df.groupby("Order_Date")["Quantity"].sum().reset_index()
sales_agg.columns = ["ds", "y"]

# Train Prophet model
model = Prophet()
model.fit(sales_agg)

# Save the model
joblib.dump(model, "/mnt/data/prophet_model.pkl")

# Load the saved model
loaded_model = joblib.load("/mnt/data/prophet_model.pkl")

# Create future dataframe for forecasting
future = loaded_model.make_future_dataframe(periods=30)
forecast = loaded_model.predict(future)

# Connect to MySQL
conn = mysql.connector.connect(
    host="your_mysql_host", 
    user="your_mysql_user", 
    password="your_mysql_password", 
    database="your_database"
)
cursor = conn.cursor()

# Create table if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS sales_forecast (
    id INT AUTO_INCREMENT PRIMARY KEY,
    date DATE,
    forecast_value FLOAT
);
""")

# Insert forecasted data into MySQL
for index, row in forecast.iterrows():
    cursor.execute("INSERT INTO sales_forecast (date, forecast_value) VALUES (%s, %s)", (row["ds"], row["yhat"]))

# Commit and close connection
conn.commit()
cursor.close()
conn.close()
print("Forecast results stored in MySQL successfully!")


In [ ]:
import pandas as pd
sales_df = pd.read_csv("SALES.csv")

In [ ]:
pip install prophet


In [4]:
!pip install scikit-learn


In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import OneClassSVM
import mysql.connector

# Load inventory dataset
inventory_df = pd.read_csv("Inventory1.csv")

# ✅ Use "Current_Stock" instead of "Stock_Level"
X = inventory_df[["Current_Stock"]].values  

# Train One-Class SVM model
oc_svm = OneClassSVM(nu=0.05, kernel="rbf", gamma=0.1)
oc_svm.fit(X)

# Predict anomalies (-1 = anomaly, 1 = normal)
inventory_df["Anomaly"] = oc_svm.predict(X)

# Store results in MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="VIJAY@25",
    database="SmartInventory"
)
cursor = conn.cursor()

# Create table if not exists
cursor.execute("""
CREATE TABLE IF NOT EXISTS inventory_anomalies (
    id INT AUTO_INCREMENT PRIMARY KEY,
    product_id INT,
    stock_level FLOAT,
    anomaly_status INT
);
""")

# Insert anomaly detection results into MySQL
for index, row in inventory_df.iterrows():
    cursor.execute("INSERT INTO inventory_anomalies (product_id, stock_level, anomaly_status) VALUES (%s, %s, %s)", 
                   (row["Product_Key"], row["Current_Stock"], row["Anomaly"]))

# Commit and close connection
conn.commit()
cursor.close()
conn.close()
print("✅ Anomaly detection results stored in MySQL successfully!")



✅ Anomaly detection results stored in MySQL successfully!


In [2]:
import joblib

# Save the model
joblib.dump(oc_svm, "one_class_svm_model.pkl")
print("Model saved successfully!")


Model saved successfully!


In [3]:
from IPython.display import FileLink

# Create a link to download the file
FileLink("one_class_svm_model.pkl")


C:\Users\vijayabalan\one_class_svm_model.pkl